# OPEN WEATHER ONE CALL API FOR WEATHER VARIABLES
---
---

In [1]:
# https://openweathermap.org/api/one-call-3
# Lesson 5.01 - APIs
# https://note.nkmk.me/en/python-unix-time-datetime/

# Imports
---

In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests

import datetime

%matplotlib inline
pd.set_option('display.max_columns', None)

# Testing: Historical Daily Weather from One Call API
---

In [3]:
url_hist_daily = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658362065&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
hist_daily_req = requests.get(url_hist_daily)
print(f'Status: {hist_daily_req.status_code}')

wd_hist = hist_daily_req.json()
print(wd_hist.keys())

wd_hist_df = json_normalize(wd_hist['data'])
wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_hist_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'data'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,2022-07-20 17:07:45,312.89,311.34,1014,17,283.27,0,3.6,220


In [4]:
url_hist_point = "https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=38.77346634864807&lon=-121.36368582956493&dt=1658363457&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
hist_point_req = requests.get(url_hist_point)
print(f'Status: {hist_point_req.status_code}')

wd_hist = hist_point_req.json()
print(wd_hist.keys())

wd_hist_df = json_normalize(wd_hist['data'])
wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

# Convert dt UNIX to datetime UTC
wd_hist_df['dt'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_hist_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'data'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg
0,2022-07-20 17:30:57,312.54,310.9,1014,17,282.99,0,3.6,200


## Function hist_weather(unix_datetime)

In [5]:
def hist_weather(unix_datetime, lat, lon):
    '''
    Using the OpenWeather OneCall 3.0 API to pull historical daily data
    Input: UNIX datetime for day requesting
    Output: Pandas dataframe for that day's weather
    '''
    
    url_hist_point = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={unix_datetime}&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
    hist_point_req = requests.get(url_hist_point)
    print(f'API Status: {hist_point_req.status_code}')

    wd_hist = hist_point_req.json()

    wd_hist_df = json_normalize(wd_hist['data'])
    wd_hist_df = wd_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
           'dew_point', 'clouds', 'wind_speed', 'wind_deg']]

    # Convert dt UNIX to datetime UTC
    wd_hist_df['datetime'] = wd_hist_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_hist_df

# Testing: Current/7-Day Forecast Daily Weather from One Call API

In [6]:
url_current_daily = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,hourly,alerts&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
current_daily_req = requests.get(url_current_daily)
print(f'Status: {current_daily_req.status_code}')

wd_current = current_daily_req.json()
print(wd_current.keys())

wd_current_df = json_normalize(wd_current['daily'])
wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# TEMPERATURE - KELVINS
temp_list = []
for i in range(len(wd_current['daily'])):
    avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
    temp_list.append(avg_temp)
wd_current_df['temp_avg'] = temp_list

# FEELS LIKE - KELVINS
fl_list = []
for i in range(len(wd_current['daily'])):
    avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
    fl_list.append(avg_fl)
wd_current_df['feels_like_avg'] = fl_list

# Convert dt UNIX to datetime UTC
wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wd_current_df.head(10)

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'daily'])


,dt,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,wind_gust,temp_avg,feels_like_avg,datetime
0,1659902400,1014,26,282.74,0,4.76,193,7.38,298.84,298.21,2022-08-07 13:00:00
1,1659988800,1013,21,280.18,1,4.67,195,6.27,298.63,297.32,2022-08-08 13:00:00
2,1660075200,1012,24,280.99,2,5.48,213,6.56,298.32,296.84,2022-08-09 13:00:00
3,1660161600,1015,21,278.80,0,4.42,211,6.96,298.53,297.19,2022-08-10 13:00:00
4,1660248000,1014,16,277.77,0,4.30,203,7.05,300.53,298.93,2022-08-11 13:00:00
5,1660334400,1012,12,275.10,0,3.71,192,6.76,300.61,298.81,2022-08-12 13:00:00
6,1660420800,1011,11,273.93,0,4.09,181,6.69,301.06,299.25,2022-08-13 13:00:00
7,1660507200,1011,11,274.63,0,3.30,190,5.87,301.61,299.99,2022-08-14 13:00:00


## Function curr_weather(lat, lon):

In [7]:
def curr_weather(lat, lon):
    '''
    Using OpenWeather OneCall 3.0 API, takes in the latitude and longitude of a location and returns a dataframe with current + 7-day forecast.
    Input: latitude and longitude of a location
    Output: dataframe
    '''
    
    url_current_daily = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
    current_daily_req = requests.get(url_current_daily)
    print(f'Status: {current_daily_req.status_code}')

    wd_current = current_daily_req.json()
    print(wd_current.keys())

    wd_current_df = json_normalize(wd_current['daily'])
    wd_current_df = wd_current_df[['dt', 'pressure', 'humidity', 'dew_point',
           'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

    # TEMPERATURE - KELVINS
    temp_list = []
    for i in range(len(wd_current['daily'])):
        avg_temp = round(sum([value for value in wd_current['daily'][i]['temp'].values()])/len(wd_current['daily'][i]['temp']), 2)
        temp_list.append(avg_temp)
    wd_current_df['temp_avg'] = temp_list

    # FEELS LIKE - KELVINS
    fl_list = []
    for i in range(len(wd_current['daily'])):
        avg_fl = round(sum([value for value in wd_current['daily'][i]['feels_like'].values()])/len(wd_current['daily'][i]['feels_like']), 2)
        fl_list.append(avg_fl)
    wd_current_df['feels_like_avg'] = fl_list

    # Convert dt UNIX to datetime UTC
    wd_current_df['datetime'] = wd_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    return wd_current_df

# Hourly - Need Paid Subscription for Historical Hourly

## Testing: Current/2-Day Forecast Hourly Weather from One Call API
---

In [8]:
url_current_hourly = "https://api.openweathermap.org/data/3.0/onecall?lat=38.752125&lon=-121.288010&exclude=current,minutely,daily&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
current_hourly_req = requests.get(url_current_hourly)
print(f'Status: {current_hourly_req.status_code}')

wh_current = current_hourly_req.json()
print(wh_current.keys())

wh_current_df = json_normalize(wh_current['hourly'])
wh_current_df = wh_current_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point',
       'clouds', 'wind_speed', 'wind_deg', 'wind_gust']]

# Convert dt UNIX to datetime UTC
wh_current_df['datetime'] = wh_current_df['dt'].apply(lambda x: datetime.datetime.fromtimestamp(x))

wh_current_df.head()

Status: 200
dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'hourly'])


,dt,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,wind_gust,datetime
0,1659924000,304.73,303.72,1011,32,286.04,0,4.76,193,5.26,2022-08-07 19:00:00
1,1659927600,303.99,302.81,1011,31,284.92,0,4.56,192,7.38,2022-08-07 20:00:00
2,1659931200,302.02,301.16,1012,34,284.60,0,3.78,172,6.24,2022-08-07 21:00:00
3,1659934800,299.58,299.58,1013,37,283.73,0,3.57,159,5.88,2022-08-07 22:00:00
4,1659938400,296.67,296.20,1013,43,283.38,0,3.65,150,6.09,2022-08-07 23:00:00


## Testing: Historical Hourly Weather from History API

In [9]:
'''
url_hist_hourly = "http://history.openweathermap.org/data/2.5/history/city?lat=38.752125&lon=-121.288010&type=hour&start=1659470400&end=1659470400&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"
hist_hourly_req = requests.get(url_hist_hourly)
print(f'Status: {hist_hourly_req.status_code}')

wh_hist = hist_hourly_req.json()
print(wh_hist.keys())

wh_hist_df = json_normalize(wh_hist['data'])
wh_hist_df = wh_hist_df[['dt', 'temp', 'feels_like', 'pressure', 'humidity',
       'dew_point', 'clouds', 'wind_speed', 'wind_deg',
       'wind_gust']]
wh_hist_df.head()
'''

'\nurl_hist_hourly = "http://history.openweathermap.org/data/2.5/history/city?lat=38.752125&lon=-121.288010&type=hour&start=1659470400&end=1659470400&appid=2f01a2c2a7f1ffcb24807b893d1ba1d2"\nhist_hourly_req = requests.get(url_hist_hourly)\nprint(f\'Status: {hist_hourly_req.status_code}\')\n\nwh_hist = hist_hourly_req.json()\nprint(wh_hist.keys())\n\nwh_hist_df = json_normalize(wh_hist[\'data\'])\nwh_hist_df = wh_hist_df[[\'dt\', \'temp\', \'feels_like\', \'pressure\', \'humidity\',\n       \'dew_point\', \'clouds\', \'wind_speed\', \'wind_deg\',\n       \'wind_gust\']]\nwh_hist_df.head()\n'